In [2]:
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np
import pandas as pd
import spacy
import torch
from google.cloud import bigquery
import os

In [3]:
os.environ["GOOGLE_APPLICATION_CREDENTIALS"] = "../ETL/credencial.json"
client = bigquery.Client()

In [4]:
def consulta(query):
    query_job = client.query(query)
    results = query_job.result()
    
    rows = []
    for row in results:
        rows.append(row)

    # Verifica si hay filas antes de crear el DataFrame
    if rows:
        df = pd.DataFrame(data=[list(x.values()) for x in rows], columns=list(rows[0].keys()))
        return df

In [5]:
db = 'restaurant_staging_dataset'
tb_names = 'restaurants_dim'
tb_geo = 'geographical_data_dim'
tb_cat_google ='category_google_dim'
tb_cat_yelp = 'category_yelp_dim'
tb_atr_google = 'attributes_google_dim'
tb_atr_yelp = 'attributes_yelp_dim'

In [6]:
query = f"""
    SELECT
            name.name,
            geo.latitude,
            geo.longitude
    FROM
    `{db}.{tb_names}` AS name
    JOIN
    `{db}.{tb_geo}` AS geo
    ON
    name.business_id = geo.business_id;
    """

df_main = consulta(query)
df_main.head()

,name,latitude,longitude
0,$,26.348095,-80.084331
1,.,42.138498,-87.962940
2,M,40.798193,-74.014895
3,JW,47.337044,-122.593467
4,PT,40.712621,-73.962286


In [21]:
query = f"""
SELECT *
FROM `{db}.{tb_cat_yelp}`;
"""
df_category_yelp = consulta(query)

query = f"""
SELECT *
FROM `{db}.{tb_cat_google}`;
"""
df_category_google = consulta(query)

In [22]:
df_category_google.head()

,business_id,American_restaurant,Asian_restaurant,Barbecue_restaurant,Breakfast_restaurant,Caribbean_restaurant,Chicken_restaurant,Chicken_wings_restaurant,Chinese_restaurant,Dessert_restaurant,...,Peruvian_restaurant,Pizza_restaurant,Salvadoran_restaurant,Seafood_restaurant,Soul_food_restaurant,Sushi_restaurant,Taco_restaurant,Thai_restaurant,Vegan_restaurant,Vietnamese_restaurant
0,0x40771faa13e3c185:0xec402d308a3cbf48,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,0x4cb555faaafddd59:0x5800723c79470569,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,0x4cc988052259d771:0x7e8ad4ae56f776a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,0x4d352ae3b0e17efd:0xf0f6d5e6b27a7e4a,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,0x4d4d1c7203ed6cbd:0x2799a2f45daace0c,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [23]:
df_category_google['American_restaurant'].unique()

array([nan,  0.,  1.])

In [9]:
df_category_yelp.head()

,business_id,pizza,sandwiches,fast food,american traditional,mexican,nightlife,breakfast & brunch,burgers,bars,...,salad,cafes,delis,japanese,sushi bars,barbeque,diners,asian fusion,steakhouses,caterers
0,MTSW4McQd7CbVtyjqoe9mw,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,9OG5YkX1g2GReZM0AskizA,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,tMkwHmWFUEXrC9ZduonpTg,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0
3,2xVsWBNFwZOxIOdd9Mwnww,0,0,0,0,0,0,0,1,0,...,0,0,0,0,0,0,0,0,0,0
4,wghnIlMb_i5U46HMBGx9ig,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
len(df_category_yelp)

15305

In [24]:
def norm(df):
    # Filtrar las columnas que tienen valor 1 para cada fila
    df_filtered = df.iloc[:, 1:].apply(lambda x: df.columns[1:][x == 1].tolist(), axis=1)

    # Crear un nuevo DataFrame con las columnas "id" e "lista_nombres"
    df_result = pd.DataFrame({
        "business_id": df["business_id"],
        "category": df_filtered
    })
    return df_result

In [18]:
df_category_yelp = norm(df_category_yelp)
df_category_yelp.head()

,business_id,category
0,MTSW4McQd7CbVtyjqoe9mw,[coffee & tea]
1,9OG5YkX1g2GReZM0AskizA,[italian]
2,tMkwHmWFUEXrC9ZduonpTg,"[seafood, japanese]"
3,2xVsWBNFwZOxIOdd9Mwnww,[burgers]
4,wghnIlMb_i5U46HMBGx9ig,[chinese]


In [25]:
df_category_google = norm(df_category_google)
df_category_google.head()

,business_id,category
0,0x40771faa13e3c185:0xec402d308a3cbf48,[]
1,0x4cb555faaafddd59:0x5800723c79470569,[]
2,0x4cc988052259d771:0x7e8ad4ae56f776a,[]
3,0x4d352ae3b0e17efd:0xf0f6d5e6b27a7e4a,[]
4,0x4d4d1c7203ed6cbd:0x2799a2f45daace0c,[]


***

In [8]:
query = f"""
SELECT *
FROM `{db}.{tb_atr_yelp}`;
"""
df_attributes_yelp = consulta(query)

query = f"""
SELECT *
FROM `{db}.{tb_atr_google}`;
"""
df_attributes_google = consulta(query)